# 로케이션 할당
- input data는 /input folder에 넣어주세요. 
- 결과물들은 각 단계마다 /output folder에서 보실 수 있습니다.
- location_assignment.py 폴더와 같은 경로에서 사용하셔야 합니다. 

### 작업 순서 설명

> (선택)으로 표시되어 있는 부분은 진행하지 않으셔도 됩니다! 상황에 따라 필요하시다면 진행해주시면 됩니다!

0. **모듈 호출**
1. **column 값 확인**
2. 제거 ~~**박스 필터링**~~
3. **랭크 필터링** (선택)
4. **plt 분리** (선택)
5. **zone 할당**

### 0. 모듈 호출 및 데이터 저장
- 코드를 돌리기 위해 필요한 내용을 호출합니다.
    - 모든 실행은 `shift` + `enter`나 재생 버튼을 눌러주시면됩니다.
- 작업하고자 하시는 데이터 저장은 로케이션 할당 폴터의 `input` 폴더에 진행해주세요
    - path 부분에 파일명을 작성해주시면 됩니다.
    - raw data는 엑셀에서 csv로 추출해서 넣어주시는게 제일 좋습니다.

In [1]:
from location_assignment import *
import pandas as pd
import numpy as np
import random
import sys
import os
import openpyxl
import warnings
warnings.filterwarnings("ignore")

seed = 42
random.seed(seed)

input뒤에 파일명을 넣어주시면 됩니다.
- 경로가 입력되지 않으면, input 파일의 예시 파일(LA_givendata_sample.csv)을 통해 작업합니다.
- 또한, 공백이 없는 편이 안전합니다.

In [2]:
path = "input/zet.csv"
raw_df = check_df(path)
pre_df = raw_df.copy()
pre_df.head()

,파일구분,주문유형1,주문유형2,월구분,배정일,배송일,창고,운송장번호,주문번호,new_주문번호,상품코드,Rank,온도대구분자,온도대,상품명,수량,엑셀상품명,엑셀옵션명,비고
0,제트용인,합포,합포,3,20240303,20240303,상온창고,584152654855,2402291404376410,202403032402291404376410,P001421,A,NaN,상온,닥터퍼피365 프리미엄 STRONG,2,P001421,[닥터퍼피365] 프리미엄 반려동물 관절＆뼈 영양제 STRONG (2개단위 구매가능),NaN
1,제트용인,합포,합포,3,20240303,20240303,상온창고,584152654855,2402291404376410,202403032402291404376410,E001396,C,NaN,상온,[휴홈] 건조기용 바닐라코튼향 섬유유연제(30매),1,E001396,[타임특가] [휴홈] 개별포장 건조기용 섬유유연제 바닐라코튼/화이트머스크향(30매입...,NaN
2,제트용인,합포,합포,3,20240303,20240303,상온창고,584152654855,2402291404376410,202403032402291404376410,H001279,A,NaN,상온,김오곤 관절보MSM 1100mg*30정,2,H001279,"[떠리데이] 김오곤 관절보MSM 건강기능식품 엠에스엠 100프로 1,100mg*30정",NaN
3,제트용인,합포,합포,3,20240303,20240303,상온창고,584152654855,2402291404376410,202403032402291404376410,C012354,B,NaN,상온,[트루비] 헬프미 모이스쳐 앰플 30ml,1,PTA240131147001S02P021,[트루비] 헬프미 앰플 30ml 3종 택1 ＋ 사은품(샘플)증정/선택 : 모이스쳐 앰플,NaN
4,제트용인,합포,합포,3,20240303,20240303,상온창고,584152654855,2402291404376410,202403032402291404376410,C012360,A,NaN,상온,[트루비] 헬프미 앰플 샘플,20,PTA240131147003S02P021,[트루비] 헬프미 앰플 30ml 3종 택1 ＋ 사은품(샘플)증정/선택 : 링클케어 앰플,NaN


### 1. column 값 확인
- 아래의 코드를 돌리면, 시스템에서 인식한 column 값이 나오게 됩니다.
    - 임의로 내용을 입력해도 좋지만, 아래 코드의 결과물을 복사, 붙여넣기해서 진행하시는게 좋습니다.

In [3]:
# pre_df's columns
pre_df.columns

Index(['파일구분', '주문유형1', '주문유형2', '월구분', '배정일', '배송일', '창고', '운송장번호', '주문번호',
       'new_주문번호', '상품코드', 'Rank', '온도대구분자', '온도대', '상품명', '수량', '엑셀상품명',
       '엑셀옵션명', '비고'],
      dtype='object')

공통 선택 부분입니다.(해당 부분은 무조건 있어야 하는 데이터입니다.)
- 하단의 assert ~ 부분은 column이 있는지 체크하는 코드입니다.
    - 아무런 결과값이 없다면 아래로 넘어가시면 됩니다.
    - AssertionError가 발생했다면, 화살표가 표시된 column 값에 오타가 있는 상황입니다. 수정해주세요
- 하단의 각 값은 아래와 같습니다.
    - zone_num: 존의 개수입니다.
    - trial_row: 크기순 정렬될 상품의 row 수입니다. 예를 들어 zone_num이 10이고, 3이라면 30개가 크기순으로 정렬되고, 이후는 랜덤으로 배치합니다.    
        - 유의미하게 큰 데이터를 생각해 넣어주시는게 좋습니다.
    - error_percentage: 오차범위에 대한 상,하한입니다. 해당하는 비율만큼 오차를 허용합니다.
        - % 변경을 위해서는 100을 곱해서 생각해주세요
    - upper_limit: zone 하나에 들어갈 수 있는 최대 품목수입니다.
        - zone과 함께 계산했을 때, 불가능한 값이 나오면 5. zone 할당에서 알려줍니다.
        - 이 경우, upper_limit을 늘리거나 zone_num을 늘려주시면 됩니다.

In [4]:
order_col_name = "주문번호" #주문번호
assert order_col_name in pre_df.columns

sku_col_name = "상품코드" # "Sku Code" #품목코드
assert sku_col_name in pre_df.columns

pcs_col_name = "수량" #작업량
assert pcs_col_name in pre_df.columns

width = 80
zone_num = 5
trial_row = 3
error_percentage = 0.05

upper_limmit = 24

Rank 설정을 위해 필요한 내용입니다.
- 한개만 진행시: ["A"]
- 여러개 진행시: ["A", "B"]

In [5]:
rank_col_name = "Rank"
assert rank_col_name in pre_df.columns

filter_rank = ["A"]
assert pre_df[rank_col_name].isin(filter_rank).any(), f"No rows with {rank_col_name} in {filter_rank}"

### 2. Rank Filtering(선택)
- 위에서 설정한 rank 별로 필터링합니다.
    - 해당 부분을 돌린 후에는 Rank에 대해 필터링된 파일을 기준으로 작업하게 됩니다. 
    - 필터링된 값은 `_rank_filtered`라는 이름으로 output folder에 저장됩니다.
- 없는 상태에서 돌리게 되면 에러가 나옵니다.

In [6]:
file_name = path.split('/')[-1].split('.')[0] + '_rank_filtered.csv'
rank_filtered_df = pre_df[pre_df[rank_col_name].isin(filter_rank)]
rank_filtered_df.to_csv('output/' + file_name, index=False)
pre_df = rank_filtered_df.copy()

### 4. PLT Filtering

> 만약 PLT로 이미 나누어져 있다면, 아래 plt_cut을 100이상으로 설정하시면 됩니다.

두가지 방법이 존재합니다.
1. 우선 아래에서 정한 PLT 범위에 따라서 시스템에서 자동으로 filtering을 해줍니다.
    - 이때, 30이라면 zone 별로 할당된 작업량의 30%이상에 대해서 필터링한다는 의미입니다.
        - 예시로 10개의 존으로 나눈후, 30이라면 전체의 3%이상의 작업량이 할당되어 있기 때문에, PLT로 빼는 것이 이후 작업량 균등할당을 위해서도 좋기 때문에 해당하는 부분으로 진행되어 있습니다.
    - 이는 `_PLT_filtered`라는 이름으로 output folder에 저장됩니다.
2. csv 파일로 가셔서, 수동으로 설정하실 PLT 그룹을 설정한뒤 하단의 부분을 실행시켜주시면, 수동으로 설정한 데이터를 불러와서 작업하게 됩니다.
    - 꼭 "PLT"로 저장해주세요
    - 단, 시스템에서 돌아가는 PATH 값을 인식하고 있기 때문에, 해당하는 페이지를 끄시게 되면 다 다시 작업하셔야되다보니, 주의해주세요

- 주요) PLT filtering은 pcs수를 기준으로 처리하지만, zone 분할을 sku 수를 기준으로 처리합니다.

In [7]:
plt_cut = 100

In [8]:
df_grouped = group_order_pcs(pre_df, pcs_col_name, sku_col_name)
df_grouped, df_plt, total_psc, total_order, length = cutting_plt(df_grouped, sku_col_name, plt_cut, zone_num)
df_grouped['zone 할당'] = [0 for _ in range(len(df_grouped))]
# concat df_plt and df_grouped
df_grouped = pd.concat([df_grouped, df_plt], axis=0)
df_grouped['zone 할당'] = df_grouped['zone 할당'].astype(str)
file_name = path.split('/')[-1].split('.')[0] + '_plt_filtered.csv'
df_grouped.to_csv('output/' + file_name, index=False)

### 5. Zone 할당
- 위에서 저장된 `output/{filename}_plt_filtered.csv`를 불러오기를 우선시도하고 진행합니다.

In [9]:
file_name = path.split('/')[-1].split('.')[0] + '_plt_filtered.csv'
pre_zone_df = pd.read_csv('output/' + file_name)
if "PLT" not in pre_zone_df['zone 할당'].unique():
    pre_zone_df['zone 할당'] = pre_zone_df['zone 할당'].astype(int).astype(str)
df_grouped = pre_zone_df[pre_zone_df['zone 할당'] == '0']
df_grouped.drop(['zone 할당'], axis=1, inplace=True)
df_plt = pre_zone_df[pre_zone_df['zone 할당'] == "PLT"]

In [10]:
print(f"length of df_grouped: {len(df_grouped)}")
print(f"length of df_plt: {len(df_plt)}")

length of df_grouped: 112
length of df_plt: 0


- upper_limit: zone 하나에 들어갈 수 있는 최대 품목수입니다.(이때 plt filtering을 했다면, plt filtering된 품목은 빠진 상태입니다.)
    - zone과 함께 계산했을 때, 불가능한 값이 나오면 아래에서 알려줍니다.
    - 이 경우, upper_limit을 늘리거나 zone_num을 늘려주시면 됩니다.
- 별 이상이 없다면 그냥 진행하셔도 좋습니다.

In [11]:
if int(length/zone_num) > upper_limmit:
    print(f"length/zone_num: {length/zone_num} > {upper_limmit}")
    print("Please increase zone_num or decrease plt_cut")
else:
    print(f"length/zone_num: {length/zone_num} < {upper_limmit}")
    print("You can proceed")

length/zone_num: 22.4 < 24
You can proceed


In [13]:
df_grouped = zone_assignment(df_grouped, sku_col_name, zone_num, trial_row, error_percentage, total_psc, total_order, length, upper_limmit)

zone_df = pd.concat([df_plt, df_grouped], ignore_index=True)
file_name = path.split('/')[-1].split('.')[0] + '_grouped_zone.csv'
zone_df.to_csv('output/' + file_name, index=False)

final_df = merge_zone(pre_df, zone_df, sku_col_name)
if "index" in final_df.columns:
    final_df.drop(columns=["index"], inplace=True)
file_name = path.split('/')[-1].split('.')[0] + '_final.csv'
final_df.to_csv('output/' + file_name, index=False)

standard_pcs: 33539.6, standard_order: 11376.8, standard_sku_count: 22
1's exisiting order: 3425, exisiting pcs: 11349
upper_limit_order: 8520.64, lower_limit_order: 7382.959999999999
upper_limit_sku: 21, lower_limit_sku: 17
selected_indices' sum:  7803
2's exisiting order: 3595, exisiting pcs: 13220
upper_limit_order: 8350.64, lower_limit_order: 7212.959999999999
upper_limit_sku: 21, lower_limit_sku: 17
selected_indices' sum:  7407
3's exisiting order: 3977, exisiting pcs: 7085
upper_limit_order: 7968.639999999999, lower_limit_order: 6830.959999999999
upper_limit_sku: 21, lower_limit_sku: 17
selected_indices' sum:  7261
4's exisiting order: 5007, exisiting pcs: 9968
upper_limit_order: 6938.639999999999, lower_limit_order: 5800.959999999999
upper_limit_sku: 21, lower_limit_sku: 17
selected_indices' sum:  6653
5's exisiting order: 5753, exisiting pcs: 12259
upper_limit_order: 6192.639999999999, lower_limit_order: 5054.959999999999
upper_limit_sku: 21, lower_limit_sku: 17
selected_indice

### 6. 엑셀로 추출
- 최종적으로 엑셀로 추출하는 함수입니다.

In [15]:
# try to read zone_grouped.csv
file_name = path.split('/')[-1].split('.')[0] + '_grouped_zone.csv'
try:
    f_zone_grouped_df = pd.read_csv('output/' + file_name)
    print("grouped_zone.csv is successfully read.")
except:
    f_zone_grouped_df = None
    print("grouped_zone.csv is not found.")

grouped_zone.csv is successfully read.


In [16]:
base_dir = 'output/'
file_name = path.split('/')[-1].split('.')[0] + '_LA.xlsx'
xlxs_dir = os.path.join(base_dir, file_name)

with pd.ExcelWriter(xlxs_dir) as writer:
    final_df.to_excel(writer, sheet_name = '요청자료')
    if f_zone_grouped_df is not None:
        f_zone_grouped_df.to_excel(writer, sheet_name = '품목_배치zone할당')